2D Example demonstrating a spill and well separated by a stream.

In [ ]:
import sys
import math
from io import StringIO
import os
import shutil
import platform
import numpy as np
from subprocess import check_output
import flopy
import pandas as pd
import matplotlib.pyplot as plt
import flopy.utils.binaryfile as bf

import config

print(np.__version__)

modelpth = os.path.join('Cara_model')
modelname = 'Spill'
mfexe = 'mfnwt'
mtexe = 'mt3dusgs'
if platform.system() == 'Windows':
    mfexe += '.exe'
    mtexe += '.exe'

# Instantiate MODFLOW model
mf = flopy.modflow.Modflow(modelname, version='mfnwt', exe_name=mfexe, 
                           model_ws=modelpth)

# Output Control: Create a flopy output control object
oc = flopy.modflow.ModflowOc(mf, stress_period_data={(0,0):['save head','save budget']})

# Newton-Rhapson Solver: Create a flopy nwt package object
headtol = 0.0001
fluxtol = 0.06
maxiterout = 200
thickfact = 1E-005
linmeth = 2
iprnwt = 1
ibotav = 0
nwt = flopy.modflow.ModflowNwt(mf, headtol=headtol, fluxtol=fluxtol, maxiterout=maxiterout,
                               thickfact=thickfact, linmeth=linmeth, iprnwt=iprnwt, ibotav=ibotav,
                               options='COMPLEX')



Since this is a hyporthetical model, we will generate ground surface from some some equations

In [ ]:
# Note that defining the ground surface this way allows to rediscretize on the fly if needed.
def topElev_Center(x,y):
    return ((-0.003 * x) + 260.) + (((-2E-9 * (x - 5000.)) + 1E-5) * (y + 1500.)**2)

# Model domain and grid definition
Lx = 10000.
Ly = 3000.
nlay = 1
nrow = 30
ncol = 100

delr = Lx / ncol
delc = Ly / nrow

xmax = ncol * delr
ymax = nrow * delc

X, Y = np.meshgrid(np.linspace(delr / 2, xmax - delr / 2, ncol),
                   np.linspace(ymax - delc / 2, 0 + delc / 2, nrow))

ibound = np.ones((nlay, nrow, ncol))

# modify the y values a bit
Y_mm = -1* np.flipud(Y)
topElev = topElev_Center(X, Y_mm)
botElev = np.zeros(topElev.shape)
strtElev= topElev - 1.

Steady = [True,False,False]
nstp = [1, 100, 200]
tsmult = [1, 1, 1]
perlen = [1, 36525, 73050]
    
# Create the discretization object
dis = flopy.modflow.ModflowDis(mf, nlay, nrow, ncol, nper=3, delr=delr, delc=delc,
                               top=topElev, botm=botElev, laycbd=0, itmuni=4, lenuni=1,
                               steady=Steady, nstp=nstp, tsmult=tsmult, perlen=perlen)

Now instatiate the UPW package

In [ ]:
laytyp = 1
layavg = 0
chani = 1.0
layvka = 0
hdry = -2e+20
iphdry = 1
hk = 20.
hani = 1
vka = 0.01
ss = 0.00001
sy = 0.20

upw = flopy.modflow.ModflowUpw(mf, laytyp=laytyp, layavg=layavg, chani=chani, layvka=layvka,
                               ipakcb=53, hdry=hdry, iphdry=iphdry, hk=hk, hani=hani,
                               vka=vka, ss=ss, sy=sy)

# Instantiate BAS
bas = flopy.modflow.ModflowBas(mf, ibound=ibound, hnoflo=hdry, strt=strtElev)

Now work on instantiating the boundary conditions

In [ ]:
# Recharge Package: The total volume coming in from the upper and lower rows should 
# total 2000.0 on each side (4,000 total)
rech = np.zeros_like(topElev)
individual_cell_rate = 50000. / ncol / (delr*delc)
rech[0,:] = rech[-1,:] = individual_cell_rate
irch = np.zeros_like(rech)  # Remember, 0-based silliness in use
   
rch = flopy.modflow.ModflowRch(mf, nrchop=1, ipakcb=53, rech=rech, irch=irch)

In [ ]:
# Evapotranspiration Package: The ET rate is 0.003 everywhere in the model, so rediscretization
# doesn't matter, can keep a constant rate.
# nevtop = 1
# evtr = 0.003
# surf = topElev
# exdp = 15
# evt = flopy.modflow.ModflowEvt(mf, nevtop=nevtop, ipakcb=53, surf=surf, evtr=evtr, exdp=exdp)

In [ ]:
# Streamflow Routing Package: Try and set up with minimal options in use
# 9 11 IFACE # Data Set 1: ISTCB1  ISTCB2
nstrm = 100
nss = 1
const = 1.283833650957E+005
dleak = 0.0001
istcb1 = 53
istcb2 = 0
isfropt = 0
nstrail = 10
isuzn = 1
nsfrsets = 30
irtflg = 1
numtim = 2
weight = 0.75
flwtol = 0.0001
segment_data = None
channel_geometry_data = None
channel_flow_data = None
dataset_5 = None
reachinput = False

# The next couple of lines set up the reach_data for the 30x100 hypothetical model.
# Will need to adjust the row based on which grid discretization we're doing.
# Ensure that the stream goes down one of the middle rows of the model.

# Determine the middle row and store in rMid (account for 0-base)
rMid = nrow//2 - 1

s1 = "k,i,j,iseg,ireach,rchlen,iface\n"
for y in range(ncol):
    #    layer +    row    +        col   +  iseg +   irch      +     rchlen     +    iface
    s1 += '0,' + str(rMid) + ',' + str(y) + ',1,' + str(y+1) + ',' + str(delr) + ',' + '0\n'

s1 = StringIO(s1)
reach_data = np.genfromtxt(s1, delimiter=',',names=True, dtype=[('k', '<f8'), ('i', '<f8'), ('j', '<f8'), ('iseg', '<f8'), ('ireach', '<f8'), ('rchlen', '<f8')])

# Will need to adjust the elevations of the upper and lower ends of the segment based on topElev array
# 5 m below ground surface (5 m of stream incision)
s2 = StringIO(u"nseg,icalc,outseg,iupseg,nstrpts,flow,runoff,etsw,pptsw,roughch,roughbk,cdpth,fdpth,awdth,bwdth,hcond1,thickm1,elevup,width1,depth1,hcond2,thickm2,elevdn,width2,depth2\n1,1,0,0,0,40000.0,0.0,0.0,0.0,0.028,0.0,0.0,0.0,0.0,0.0,1.0,1.0,254.899750,20.0,0.0,1.0,1.0,225.150250,20.0,0.0")
segment_data = np.genfromtxt(s2, delimiter=',',names=True)

# Adjust the elevations of the upstream and downstream ends of the segment
segment_data['elevup'] = topElev[rMid,0] - 10.
segment_data['elevdn'] = topElev[rMid,ncol-1] - 10.

# Be sure to convert segment_data to a dictionary keyed on stress period.
segment_data = np.atleast_1d(segment_data)
segment_data = {0: segment_data,
                1: segment_data,
                2: segment_data}

# There are 2 stress periods
dataset_5 = {0: [nss, 0, 0],
             1: [nss, 0, 0],
             2: [nss, 0, 0]}

channel_flow_data = {0: {1:[[ 1.0E+01, 1.0E+00, 2.5489975E+02, 2.0E+01],    #Data set 6b: HCOND1 THICKM1 ELEVUP WIDTH1
                            [ 1.0E+01, 1.0E+00, 2.2515025E+02, 2.0E+01]]},  #Data set 6c: HCOND2 THICKM2 ELEVDN WIDTH2
                     1: {2:[[ 1.0E+01, 1.0E+00, 2.5489975E+02, 2.0E+01],    #Data set 6b: HCOND1 THICKM1 ELEVUP WIDTH1
                            [ 1.0E+01, 1.0E+00, 2.2515025E+02, 2.0E+01]]},} #Data set 6c: HCOND2 THICKM2 ELEVDN WIDTH2

sfr = flopy.modflow.ModflowSfr2(mf, nstrm=nstrm, nss=nss, const=const, dleak=dleak,
                                ipakcb=53, istcb2=0, reach_data=reach_data, dataset_5=dataset_5,
                                segment_data=segment_data, channel_flow_data=channel_flow_data)

In [ ]:
gages = [[1,int(ncol/4),61,1],[1,int(ncol/2),62,1], [1,int(ncol*3/4),63,1], [1,ncol,64,1]]
files = ['Spill.gage','Spill.gag1','Spill.gag2','Spill.gag3','Spill.gag4']
gage = flopy.modflow.ModflowGage(mf, numgage=len(gages), gage_data=gages, filenames = files)

In [ ]:
# Add a municipal pumping supply well
welu = [0, 17, 25]
weld = [0, 11, 74]
wel = flopy.modflow.ModflowWel(mf, ipakcb=53, stress_period_data = {1:[weld + [-3.5e5],
                                                                       welu + [1.5e-5]]})

In [ ]:
# Add the Link MT3D package for generating the linker file
lmt = flopy.modflow.ModflowLmt(mf, output_file_name='Spill.ftl', output_file_header='extended',
                               output_file_format='formatted', package_flows = ['sfr'])

In [ ]:
# Investigate the effects of hfb
#mxfb = nrow
#nhfbnp = nrow
#hfb_data = [[0, i, 34, i, 35, 0.0001] for i in range(nrow)]   # Layer IROW1 ICOL1 IROW2  ICOL2 Hydchr 
#hfb_data[13][5] = 1
#hfb_data[14][5] = 1
#hfb_data[15][5] = 1
#hfb = flopy.modflow.ModflowHfb(mf, nhfbnp = nhfbnp, hfb_data = hfb_data)

In [ ]:
mf.write_input()
mf.run_model()

In [ ]:
hdsobj = bf.HeadFile(os.path.join(modelpth,'{0}.hds'.format('Spill')))
totim = hdsobj.get_times()
totim
hds = hdsobj.get_data(totim=totim[0])
#hds = []
#for tm in totim:
#    hds.append(hdsobj.get_data(totim=tm))

print(hds.shape)

depth = topElev - hds[0,:,:]
print(depth.shape)
print(depth.max())
print(depth.min())


In [ ]:
hdr = ['Time', 'Stage', 'Flow', 'Depth', 'Width', 'Midpt_Flow', 'Precip', 'ET', 'Runoff']
for j in range(4):
    fname = os.path.join(modelpth, 'CrnkNic.gag' + str(j + 1))
    wha = pd.read_table(fname, header=None, skiprows=2, names=hdr, delim_whitespace=True)

    if j == 0:
        wha.drop(wha.columns.difference(['Time', 'Flow']), 1, inplace=True)
        df = wha.copy()
        df.rename(columns={'Flow': 'Flow' + str(j + 1)}, inplace=True)

    if j != 0:
        df = pd.merge(df, wha[['Time', 'Flow']], on='Time', how='left')
        df.rename(columns={'Flow': 'Flow' + str(j + 1)}, inplace=True)


df = df[['Time','Flow1','Flow2','Flow3','Flow4']]


#try:
#    plt.close('all')
#except:
#    pass

#fig = plt.figure(figsize=(15, 5), facecolor='w')
#ax = fig.add_subplot(1, 1, 1)

#lns1 = ax.plot(df['Time'], df['Flow1']*35.315/86400, 'b-', linewidth=1.0, label='1/4')
#lns2 = ax.plot(df['Time'], df['Flow2']*35.315/86400, 'k-', linewidth=1.0, label='1/2')
#lns3 = ax.plot(df['Time'], df['Flow2']*35.315/86400, 'r-', linewidth=1.0, label='3/4')
#lns4 = ax.plot(df['Time'], df['Flow2']*35.315/86400, 'g-', linewidth=1.0, label='end')

#customize plot
#ax.set_xlabel('Time, days')
#ax.set_ylabel('Flow, cfs')
#ax.set_ylim([0,20])
#ticksize = 10

# add 4 items to legend
#lns = lns1+lns2+lns3+lns4
#labs = [l.get_label() for l in lns]

#legend
#leg = ax.legend(lns,labs,
#                loc='upper left', labelspacing=0.25, columnspacing=1,
#                handletextpad=0.5, handlelength=2.0, numpoints=1)
#leg._drawFrame = False 

#plt.show()




## Setup Transport Model

In [ ]:
mt = flopy.mt3d.Mt3dms(modflowmodel=mf, modelname=modelname, model_ws=modelpth,
                       version='mt3d-usgs', namefile_ext='mtnam', exe_name=mtexe,
                       ftlfilename='Spill.ftl', ftlfree=True)

# BTN package
ncomp   =    1
lunit   =    'm'
sconc   =    0.0
prsity  =    sy
cinact  =   -1.0
thkmin  =    0.000001
nprs    =   -1
nprobs  =   10
nprmas  =   10
dt0     =  100.0
nstp    = nstp    # From DIS setup
mxstrn  = 5000
ttsmult =    1.0
ttsmax  =    0

#obs = [[0, 10, 15]]

btn = flopy.mt3d.Mt3dBtn(mt, MFStyleArr=True, DRYCell=True, lunit=lunit,
                         sconc=sconc, ncomp=ncomp, prsity=prsity, cinact=cinact,
                         thkmin=thkmin, nprs=nprs, nprobs=nprobs, 
                         chkmas=True,nprmas=nprmas, dt0=dt0,  
                         mxstrn=mxstrn, ttsmult=ttsmult, ttsmax=ttsmax)
# ADV package
mixelm =    0
percel =    1.0000
mxpart = 5000
nadvfd =    1      # (1 = Upstream weighting)

adv = flopy.mt3d.Mt3dAdv(mt, mixelm=mixelm, percel=percel, mxpart=mxpart, 
                         nadvfd=nadvfd)

#GCG solver package
mxiter =   1
iter1  = 200
isolve =   3
ncrs   =   0
accl   =   1.000000
cclose =   1.00e-06
iprgcg =   5

gcg = flopy.mt3d.Mt3dGcg(mt, mxiter=mxiter, iter1=iter1, 
                         isolve=isolve, ncrs=ncrs, accl=accl, 
                         cclose=cclose, iprgcg=iprgcg)

# SSM
# ---

# First, get a dictionary that has the SSM itype for each of the boundary types.
itype = flopy.mt3d.Mt3dSsm.itype_dict()


mxss = 3002          # maximum number of boundary conditions
ssm_data = {}
#             kss,iss,jss,css,        ITYPE
ssm_data[0] = [(weld[0], weld[1], weld[2],   0., itype['WEL']),
               (welu[0], welu[1], welu[2],   0., itype['CC'])]          #Concstant Concentration
ssm_data[1] = [(weld[0], weld[1], weld[2],   0., itype['WEL']),
               (welu[0], welu[1], welu[2], 100., itype['CC'])]
ssm_data[2] = [(weld[0], weld[1], weld[2],   0., itype['WEL']),
               (welu[0], welu[1], welu[2],   0., itype['CC'])]

ssm = flopy.mt3d.Mt3dSsm(mt, mxss=mxss, stress_period_data=ssm_data)

# SFT
# ---
dispsf = 1
coldsf = 0
seg_len = np.unique(reach_data['iseg'], return_counts=True)
obs_sf = np.cumsum(seg_len[1])
obs_sf = obs_sf.tolist()

print(seg_len)
print('value: ' + str(obs_sf))

# In the first and last stress periods, concentration at the headwater is 0.0
sf_stress_period_data = {0: [0, 0, 0.0],
                         1: [0, 0, 0.0],
                         2: [0, 0, 0.0]}

gage_output = [None, None, 'CrnkNic.sftobs']

sft = flopy.mt3d.Mt3dSft(mt, nsfinit=100, mxsfbc=100, icbcsf=81, ioutobs=82,
                         isfsolv=1, cclosesf=1.0E-5, wimp=1.0, wups=1.0, mxitersf=200, crntsf=1.0, iprtxmd=0,
                         coldsf=coldsf, dispsf=dispsf, nobssf=1, obs_sf=obs_sf,
                         sf_stress_period_data = sf_stress_period_data,
                         filenames=gage_output)



### Run MT3D-USGS

In [ ]:
mt.write_input()
mt.run_model()

In [ ]:
# plot some model attributes
fig = plt.figure(figsize=(13,13))
ax = plt.subplot(111,aspect="equal")
mm = flopy.plot.ModelMap(model=mf)
mm.plot_grid()
ax = mm.ax
mf.dis.top.plot(axes=[ax],colorbar="K m/d",alpha=0.5)
mf.wel.stress_period_data.plot(axes=[ax], kper=1)
mf.sfr.stress_period_data.plot(axes=[ax])

In [ ]:
# Load binary concentration file
ucn = bf.UcnFile(os.path.join(modelpth,'{0}.ucn'.format('mt3d001')))
totim = ucn.get_times()
conc = []
for i in totim:
    conc.append(ucn.get_data(totim=i))

conc = np.array(conc)
conc.shape

# plot the concentrations
fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(15, 5))
axes.set_aspect('equal')
cmin = conc.min()
cmax = conc.max()

# set the contour list
levels = np.linspace(cmin, cmax, 11)
levels = levels.tolist()
levels.pop(0)
levels = [0.1,1,5] + levels

ax.set_title('Layer {}'.format(1))
mm = flopy.plot.ModelMap(model=mf, ax=axes)
mm.plot_grid()
ax = mm.ax
mm.contour_array(conc[50,0,:,:], masked_values=[-1.], alpha=0.5, levels=levels, colors='red')

#quadmesh = mm.plot_array(conc[200,0,15:25,20:30], masked_values=[-1.], alpha=0.1, 
#                         vmin=cmin, vmax=cmax, cmap='jet')
mf.wel.stress_period_data.plot(axes=[ax], kper=1)
mf.sfr.stress_period_data.plot(axes=[ax])

print(os.getcwd())
fname = os.path.join('Cara_model', 'Spill.sftcobs.out')
# dt = pd.read_table(fname, header=[1])